In [2]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

In [3]:
def word_occurence_matrix(text, target=None, stop_words=None, binary=True, preprocess_text=False):  
    '''
    Output is messages x (unique) words
    
    If binary=True, then each element represents if the word is in the message or not.
    Otherwise, it represents the count of how many times that word appears in that message.
    ''' 
    if target:
        text = list(filter(lambda x : target in x, text)) #Filter comments in which target word is present
        
    preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor()   
    if stop_words:        
        stop_words = [preprocessor(word) for word in stop_words] #preprocesses stop words
    if preprocess_text:
        text = [preprocessor(msg) for msg in text] #preprocesses text
        
    #calculates word count for each message
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words=stop_words, binary=binary)
    X = vectorizer.fit_transform(text).toarray()
    
    labels = vectorizer.get_feature_names()
    
    return X, labels

### Dados

In [4]:
comments = pd.read_csv('../comentarios_sorted_votes.csv')
stop_words = [word.rstrip() for word in open('stopwords.txt')]

#Preprocesses text
preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor() #lowercase and strip accents
stop_words = [preprocessor(word) for word in stop_words]
comments['text'] = [preprocessor(msg) for msg in comments['text']]
comments['text'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) if not word in stop_words])
                    for msg in comments['text']]

N = 30 #Consider only the N most frequent words

In [8]:
comments_by_channel = []
channels = []
for channel, group in comments.groupby('uploader'):
    channel_comments = ' '.join(group['text'])
    comments_by_channel.append(channel_comments)
    channels.append(channel)

### Frequencia relativa

#### 1. Em relação ao total
Aqui, foi usada a distribuição de probabilidade conjunta das palavras e canais, i.e., $p(a_i, c_j) = P(A=a_i, C= C_j)$.

In [29]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)

X = X / X.sum()
print('Frequência relativa ao total:')
df_total = pd.DataFrame(X.T, columns = channels, index = labels)
df_total.tail()

Frequência relativa ao total:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
私も乳癌になり先生方に助けて頂き今かあり感謝,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0
賢者,0.0,0.0,0.0,0.000000,0.0,0.0,0.000001,0.0
身体に,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0
隼hayabusa,0.0,0.0,0.0,0.000000,0.0,0.0,0.000007,0.0
頑張って下さい,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0


In [6]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dias', '0.0016'), ('pra', '0.0012'), ('cheiro', '0.0012'), ('olfato', '0.0011'), ('paladar', '0.0011'), ('drauzio', '0.0011'), ('deus', '0.0011'), ('dr', '0.0009'), ('vc', '0.0009'), ('virus', '0.0009'), ('dor', '0.0009'), ('video', '0.0008'), ('vai', '0.0008'), ('pessoas', '0.0007'), ('ivermectina', '0.0007'), ('sinto', '0.0007'), ('sentir', '0.0007'), ('nada', '0.0007'), ('gosto', '0.0007'), ('to', '0.0007'), ('dia', '0.0007'), ('assim', '0.0007'), ('sintomas', '0.0007'), ('agora', '0.0006'), ('covid', '0.0006'), ('bem', '0.0006'), ('ainda', '0.0006'), ('casa', '0.0005'), ('ta', '0.0005'), ('tudo', '0.0005')]


#### 2. Em relação ao canal
Aqui, foi obtida a distribuição de probabilidade das palavras em cada canal $C_k$, i.e., $p(a_i|C_k) = P(A=a_i|C_k)$. 
Com isso, podemos fazer corretamente comparações entre os canais. Foram, então, selecionadas as palavras cuja ocorrência mais se difere entre eles.

In [30]:
X, labels = word_occurence_matrix(comments_by_channel, stop_words=stop_words, binary=False)
#Makes frequencies relative to each channel so that we have the probability distribution of words for each channel
X = (X.T / X.sum(axis=1)).T
print('Frequência relativa a cada canal:')
df_canal = pd.DataFrame(X.T, columns = channels, index = labels)
df_total.tail()

Frequência relativa a cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
私も乳癌になり先生方に助けて頂き今かあり感謝,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0
賢者,0.0,0.0,0.0,0.000000,0.0,0.0,0.000001,0.0
身体に,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0
隼hayabusa,0.0,0.0,0.0,0.000000,0.0,0.0,0.000007,0.0
頑張って下さい,0.0,0.0,0.0,0.000001,0.0,0.0,0.000000,0.0


In [8]:
#Selects only the N with higher std
word_std = X.std(axis=0)
higher_std_zipped = sorted(zip(word_std, labels, X.T), reverse=True)[:N]
word_std, labels, X_t = zip(*higher_std_zipped)

word_std_i = ['%.4f'%(x) for x in word_std]
print(f'{N} palavras com maior desvio padrão entre os canais:\n')
print(list(zip(labels, word_std_i)))

30 palavras com maior desvio padrão entre os canais:

[('dr', '0.0116'), ('fernando', '0.0115'), ('dias', '0.0083'), ('lair', '0.0077'), ('ivermectina', '0.0060'), ('you', '0.0054'), ('dor', '0.0046'), ('tomar', '0.0044'), ('fiz', '0.0042'), ('comprimidos', '0.0042'), ('sintomas', '0.0041'), ('olfato', '0.0041'), ('boa', '0.0040'), ('paladar', '0.0040'), ('cheiro', '0.0039'), ('deu', '0.0038'), ('covid', '0.0037'), ('noite', '0.0037'), ('15', '0.0036'), ('ribeiro', '0.0036'), ('parabens', '0.0035'), ('pinto', '0.0034'), ('dia', '0.0033'), ('gomes', '0.0033'), ('drauzio', '0.0030'), ('good', '0.0030'), ('positivo', '0.0029'), ('dra', '0.0029'), ('deus', '0.0028'), ('to', '0.0028')]
